
Notebook chapter 04 

Autor Pedro Daia Cardoso

---








In [1]:
# Exercício de treinamento, criacao de um modelo reconhecedor dos digitos 3 e 7

#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
from fastai.vision.all import *

matplotlib.rc('image', cmap='Greys') # para mostrar tudo cinza os vetores

     |████████████████████████████████| 719 kB 31.1 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |████████████████████████████████| 362 kB 65.4 MB/s 
     |████████████████████████████████| 4.4 MB 61.7 MB/s 
     |████████████████████████████████| 212 kB 63.6 MB/s 
     |████████████████████████████████| 1.1 MB 66.5 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 140 kB 68.5 MB/s 
     |████████████████████████████████| 596 kB 74.5 MB/s 
     |████████████████████████████████| 127 kB 74.2 MB/s 
     |████████████████████████████████| 271 kB 36.0 MB/s 
     |████████████████████████████████| 144 kB 50.9 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 6.6 MB 63.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.1

In [2]:
path = untar_data(URLs.MNIST_SAMPLE) # esse sample do reconhecedor universal só tem 3s e 7s
Path.BASE_PATH = path
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()


Podemos criar um Pytorch tensor para representar a img, é um array do numpy injetavel na GPU
>`imgTensor = tensor(Image.open(threes[0]))`

In [3]:
# Criar listas para guardar todos os setes e os tres
sevenTensors = [tensor(Image.open(i)) for i in sevens]
threeTensors = [tensor(Image.open(i)) for i in threes]

sevenStacked = torch.stack(sevenTensors).float() / 255 # divide por 255 pq imagem tem 255 pixels
threeStacked = torch.stack(threeTensors).float() / 255 # mas quando sao img como um float, valores de pixels sao entre 0 e 1


Como nos temos listas, não é util para fazer computacoes matematicas; já um tensor pode ter vários "eixos/dimensões", podemos pegar a lista de imgs e "stackar" uma em cima da outra criando uma espécie de cubo 3D, com o qual dá pra fazer os calculos

>## NOMENCLATURA IMPORTANTE 
>*   rank = dimensoes de um tensor
>*   shape = tamanho de cada dimensao


Precisamos medir proximidade de pixels (distância), para comparar os números com a média
> L2 Form: pega a MEAN do quadrado das diferencas e dps tira a raiz de tudo


 

In [4]:
a = threeStacked[0] # vamos ver a distância desse 3 para a média do 3 e pro 7

distThree = ((a - threeStacked.mean(0))**2).mean().sqrt()
distSeven = ((a - sevenStacked.mean(0))**2).mean().sqrt()

if (distThree < distSeven):
  print('é um 3')
else: 
  print('é um 7')

é um 3


##Essa já é uma BASELINE válida para o modelo, é bem simples mas é sólida.

In [5]:
# pegar a validation set
threeValidImg = (path/'valid'/'3').ls().sorted()
sevenValidImg = (path/'valid'/'7').ls().sorted()
threeValidationTensors = [tensor(Image.open(i)) for i in threeValidImg]
sevenValidationTensors = [tensor(Image.open(i)) for i in sevenValidImg]

three_stackedValidation = torch.stack(threeValidationTensors).float() / 255
seven_stackedValidation = torch.stack(sevenValidationTensors).float() / 255

def mnsit_dist(a, b):
  return ((a-b)**2).mean((-1,-2)).sqrt()
  #eixos -1 e -2 são os horizontal e vertical da img

`mnsit_dist(three_stackedValidation, stackedThrees.mean(0))` 

Essa chamada nao reclamaria de diferenca em shape, retornaria o calculo da dist para todos dentro do three_stackedValidation em relacao ao 3 ideal
>Nome disso é BROADCASTING, o pytorch altera o rank de um tensor para um cálculo com tensor de rank diferente

In [6]:
def isThree(a):
  return mnsit_dist(a, threeStacked.mean(0)) < mnsit_dist(a, sevenStacked.mean(0))

# obs: ao converter bool return pra float, é 1.0 p true e 0.0 p false
# vamos calcular a accuracy, a precisão
acc_threes = isThree(three_stackedValidation).float().mean()
acc_sevens = 1 - (isThree(seven_stackedValidation).float().mean())
acc_threes, acc_sevens

(tensor(0.9584), tensor(0.9737))

####`(tensor(0.9584), tensor(0.9737))`
Este output indica uma precisão de 95% para os 3 e 97% para os 7 bom resultado mas são só 2 numeros, e sao bem diferentes, entao precisa melhorar muito

Percebe-se que o sistema ainda não tem uma capacidade de aprendizado, uma forma de alterar parâmetros para melhorar a performance, muito menos como medir algo do tipo.

>Ao invés de abordar pela Pixel Similarity, comparando uma img com uma "imagem ideal", podemos ter uma abordagem por "gradiente"
>Com o conceito de pesos, atribuímos pesos maiores a regiões de imagem mais prováveis para certo dígito, como por exemplo em um 7, a região inferior direita teria pesos menores pois é improvável, enquanto em um 8 a mesma região teria pesos maiores.


Por exemplo, uma função:
```
def pr_eight(x,w): return (x*w).sum()
```
onde *x* é a img, e *w* é um vetor de pesos

Ou seja, procurar pela melhor configuração do vetor *w* significa melhorar o reconhecimento do sistema.












#Cálculo de um gradiente



```
def f(x): return x**2

# requires_grad_()
# ^ ^ fc p dizer ao pytorch que quero calc grads nesse ponto c/ rel. a essa var
xTensor = tensor(3.).requires_grad_() 

yt = f(xTensor)
yt.backward()
xTensor.grad
```

>output

>```
tensor(6.)
```

In [ ]:
def f(t, params):
  a,b,c = params
  return a*(t**2) + b*t + c

def mse(preds, targets): return ((preds-targets)**2).mean()